## Import Reqired Libraries

In [1]:
import os

from requests import get
from bs4 import BeautifulSoup

import urllib

import time
from datetime import datetime, timedelta

import urllib.request
from urllib.parse import urlparse

import csv
import re

import glob

#### Set base_path where previously scraped information and pictures are saved.  

In [2]:
base_path = os.path.normpath(r'C:\path\to\previously\saved\scraped\files')


#### Find the most recent file written in the directory.  Save its timestamp into folder_time. 

In [3]:
files_path = os.path.join(base_path, '*')
files = sorted(glob.iglob(files_path), key=os.path.getctime, reverse=True)
timestamp = seconds = os.path.getctime(files[0])
folder_time = datetime.fromtimestamp(timestamp)


#### Define the base urls you're using.  List all of the Craigslist sites you want to scrape in search_urls. 

In [4]:
craigslist_url = 'https://cincinnati.craigslist.org'
base_url = 'https://cincinnati.craigslist.org/search/jwa'

search_urls = ['https://akroncanton.craigslist.org/d/jewelry/search/jwa', 'https://ashtabula.craigslist.org/d/jewelry/search/jwa', 'https://athensohio.craigslist.org/d/jewelry/search/jwa', 'https://chillicothe.craigslist.org/d/jewelry/search/jwa', 'https://cincinnati.craigslist.org/d/jewelry/search/jwa', 'https://cleveland.craigslist.org/d/jewelry/search/jwa', 'https://columbus.craigslist.org/d/jewelry/search/jwa', 'https://dayton.craigslist.org/d/jewelry/search/jwa', 'https://limaohio.craigslist.org/d/jewelry/search/jwa', 'https://mansfield.craigslist.org/d/jewelry/search/jwa', 'https://sandusky.craigslist.org/d/jewelry/search/jwa','https://toledo.craigslist.org/d/jewelry/search/jwa', 'https://tuscarawas.craigslist.org/d/jewelry/search/jwa', 'https://youngstown.craigslist.org/d/jewelry/search/jwa', 'https://zanesville.craigslist.org/d/jewelry/search/jwa', 'https://bgky.craigslist.org/d/jewelry/search/jwa', 'https://eastky.craigslist.org/d/jewelry/search/jwa', 'https://lexington.craigslist.org/d/jewelry/search/jwa', 'https://louisville.craigslist.org/d/jewelry/search/jwa', 'https://owensboro.craigslist.org/d/jewelry/search/jwa', 'https://westky.craigslist.org/d/jewelry/search/jwa', 'https://bloomington.craigslist.org/d/jewelry/search/jwa', 'https://evansville.craigslist.org/d/jewelry/search/jwa', 'https://fortwayne.craigslist.org/d/jewelry/search/jwa', 'https://indianapolis.craigslist.org/d/jewelry/search/jwa', 'https://kokomo.craigslist.org/d/jewelry/search/jwa', 'https://tippecanoe.craigslist.org/d/jewelry/search/jwa', 'https://muncie.craigslist.org/d/jewelry/search/jwa', 'https://richmondin.craigslist.org/d/jewelry/search/jwa', 'https://southbend.craigslist.org/d/jewelry/search/jwa', 'https://terrehaute.craigslist.org/d/jewelry/search/jwa']


In [5]:
item_anchors = []
first_page_anchors = []

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:12.0) Gecko/20100101 Firefox/12.0'}


#### Scrape the post urls from every craigslist site listed in search_urls.  

In [13]:
for search_url in search_urls:

    response = get(search_url, headers=headers)

    search_soup = BeautifulSoup(response.text, 'html.parser')

    first_page_anchors =  search_soup.find_all("a", {"class":"result-title hdrlnk"})
    for anchor in first_page_anchors:
        item_anchors.append(anchor)

    while True:
        if search_soup.find("a",{"class":"button next"}) is None:
            break
        else:
            if search_soup.find("a",{"class":"button next"})['href'] != '':
                response = get(craigslist_url + search_soup.find("a",{"class":"button next"})['href'], headers=headers)
                search_soup = BeautifulSoup(response.text, 'html.parser')
                new_anchors = search_soup.find_all("a", {"class":"result-title hdrlnk"})
                for anchor in new_anchors:
                    item_anchors.append(anchor)
            else:
                break

#### Remove duplicate post listings

In [14]:
#make sure not to scrape the same item twice
cleaned_anchors = []
for anchor in item_anchors:
    if anchor['href'] not in cleaned_anchors:
        cleaned_anchors.append(anchor['href'])

#### Scrape the post title, item price, post location, date of the post, the item image url, and the post url of posts that were posted after the last scraping took place. 

In [15]:
records = []
for link in cleaned_anchors:
    item_response = get(link, headers = headers)
    item_soup = BeautifulSoup(item_response.text,'html.parser')

    if item_soup.find("time", {"class":"date timeago"}) is not None:
        item_raw_date = item_soup.find("time", {"class":"date timeago"})['datetime']

        date_list = item_raw_date.split('-')
        day_time = date_list[2].split('T')
        time_list = day_time[1].split(':')

        file_time = datetime(year=int(date_list[0]), month=int(date_list[1]), day=int(day_time[0]), hour=int(time_list[0]), minute=int(time_list[1]), second=int(time_list[2] ) )

        if folder_time < file_time:
            item_parse_date = item_raw_date[:-2] + ":" + item_raw_date[-2:]
            d = datetime.fromisoformat(item_parse_date)

            o = urlparse(link)
            location = o.netloc.split('.')[0]

            item_title = item_soup.find("span", {"id":"titletextonly"}).get_text()
            cleaner_title = re.sub(r'[<>:"\/\\|?*]','',item_title)
            clean_title = re.sub(r'\t',' ', cleaner_title)

            if item_soup.find("span", {"class":"price"}) is not None:
                item_price = item_soup.find("span", {"class":"price"}).get_text()
            else:
                item_price = "not listed"
            if item_soup.find("div", {"class": "slide first visible"}) is not None:
                item_image_url = item_soup.find("div", {"class": "slide first visible"}).img['src']
            else:
                item_image_url = "no image"
            item_raw_date = item_soup.find("time", {"class":"date timeago"})['datetime']
            item_parse_date = item_raw_date[:-2] + ":" + item_raw_date[-2:]
            d = datetime.fromisoformat(item_parse_date)


            records.append([clean_title, item_price, location, d.strftime("%a %b %d %I:%M %p %Z") , item_image_url, link])

#### Create new repository for post information and photos to go

In [41]:
base_repository = os.path.normpath(r'C:\path\to\new\file')

file_time = datetime.now().strftime("%b %d %Y %Hh%Mm%Ss")

new_file = os.mkdir(os.path.join(base_repository, file_time ))

os.chdir(os.path.join(base_repository, file_time))

new_repository = os.path.normpath(os.path.join(base_repository, file_time))

#new_repository = os.path.normpath(os.path.join(base_repository, datetime.now().strftime("%b %d %Y %Hh%Mm%Ss")))

#### Save post information in a csv file

In [24]:
with open('cragislist_update '+ datetime.now().strftime("%b %d %Y %Hh%Mm%Ss") + '.csv', mode='w', newline='\n', encoding="utf-8") as output_file:
    ghostwriter = csv.writer(output_file)
    ghostwriter.writerow(['Item Title', 'item price', 'location', 'post date', 'image url', 'Post url'])
    ghostwriter.writerows(records)

In [22]:
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

#### Save images to new file

In [23]:
line_count = 0
for record in records:
    if record[4] != 'no image':
        urllib.request.urlretrieve(record[4], os.path.join(new_repository, record[0] + '_' + str(line_count) + '.png') )
        line_count += 1